# 06 - CME Data Generation (LSTM & GAN)

This notebook provides tools to generate synthetic CME (Coronal Mass Ejection) time-series data using:

- An LSTM sequence generator (autoregressive)
- A simple RNN-GAN with LSTM-based generator and discriminator

It is designed to plug into the existing project structure. The notebook will:

1. Try to load CME files from `../data/` using common filenames.
2. Preprocess the data into fixed-length sequences.
3. Provide PyTorch implementations for LSTM generator and an LSTM-GAN.
4. Train on found CME data, or on a synthetic example if no data is available.

Outputs: saved models and generated CSVs under `../data/` if writable.

---


In [ ]:
# Cell 1: Imports and environment checks
import os, glob, sys, math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
print('Python', sys.version)
print('Torch', torch.__version__)
print('Notebook will look for CME files in ../data/ (relative to this notebook)')


In [ ]:
# Cell 2: Utility to find data files and load them when present
from glob import glob
DATA_DIRS = [os.path.join(os.getcwd(),'../data'), os.path.join(os.getcwd(),'./data'), '/mnt/data']
candidate_files = []
for d in DATA_DIRS:
    if os.path.isdir(d):
        candidate_files += sorted(glob(os.path.join(d, 'cme*.txt')))
        candidate_files += sorted(glob(os.path.join(d, 'cme*.csv')))
        candidate_files += sorted(glob(os.path.join(d, '*.txt')))

print('Searched directories:', DATA_DIRS)
print('Found candidate files (first 20):')
for f in candidate_files[:20]:
    print(' -', f)

# Helper to attempt loading a file as CSV-like
def try_load_file(path):
    """Try to load tabular data from path using pandas with various delimiters."""
    for sep in [',','\t',';','|',' ']:
        try:
            df = pd.read_csv(path, sep=sep, engine='python')
            if df.shape[0] > 1:
                return df
        except Exception:
            pass
    # try numpy loadtxt
    try:
        arr = np.loadtxt(path)
        return pd.DataFrame(arr)
    except Exception:
        return None

# Try loading first candidate to inspect
sample_df = None
if candidate_files:
    sample_df = try_load_file(candidate_files[0])
    print('\nLoaded sample shape:', None if sample_df is None else sample_df.shape)
    if sample_df is not None:
        display(sample_df.head())
else:
    print('\nNo candidate CME data files were found in the searched paths.')


In [ ]:
# Cell 3: If no real data, create synthetic CME-like time series for demonstration
def make_synthetic_cme(n_series=200, seq_len=128, n_features=1, random_seed=42):
    rng = np.random.RandomState(random_seed)
    data = []
    for i in range(n_series):
        t = np.linspace(0, 20, seq_len)
        # base: slow variation + burst(s)
        base = 0.5*np.sin(0.2*t + rng.randn()*0.5) + 0.1*rng.randn(seq_len)
        # add a burst simulating CME intensity/time-localized event
        center = rng.uniform(4,16)
        width = rng.uniform(0.2,2.0)
        burst = rng.uniform(0.5,3.0) * np.exp(-0.5*((t-center)/width)**2)
        seq = base + burst
        data.append(seq.reshape(seq_len, n_features))
    return np.stack(data)

if sample_df is None:
    print('Creating synthetic dataset for demo...')
    data = make_synthetic_cme(n_series=512, seq_len=128)
else:
    # convert the sample_df into sequences: flatten columns to numeric and create sliding windows
    print('Converting loaded file into sequences.\nYou may want to adapt this cell to your data format.')
    arr = sample_df.select_dtypes(include=[np.number]).values
    if arr.ndim == 1:
        arr = arr.reshape(-1,1)
    # create overlapping windows
    seq_len = 128
    step = 64
    seqs = []
    for start in range(0, max(1, arr.shape[0]-seq_len), step):
        seqs.append(arr[start:start+seq_len])
    if not seqs:
        # fallback: reshape into multiple sequences if possible
        L = arr.shape[0]
        n = max(1, L // seq_len)
        arr2 = arr[:n*seq_len].reshape(n, seq_len, arr.shape[1])
        seqs = list(arr2)
    data = np.stack(seqs)

print('Data shape (n_series, seq_len, n_features):', data.shape)


In [ ]:
# Cell 4: Dataset and preprocessing utilities
class CMEDataset(Dataset):
    def __init__(self, data_array):
        # data_array: (N, T, F)
        self.x = torch.tensor(data_array, dtype=torch.float32)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx]

# normalization helpers
def fit_scaler(data):
    mu = data.mean((0,1), keepdims=True)
    std = data.std((0,1), keepdims=True) + 1e-6
    return mu, std

def apply_scaler(data, mu, std):
    return (data - mu) / std

mu, std = fit_scaler(data)
print('mu/std shapes:', mu.shape, std.shape)
data_scaled = apply_scaler(data, mu, std)
train_loader = DataLoader(CMEDataset(data_scaled), batch_size=64, shuffle=True)


In [ ]:
# Cell 5: LSTM autoregressive generator (predict next-step) and sampling helper
class LSTMAutoRegressor(nn.Module):
    def __init__(self, n_features=1, hidden_size=64, n_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(n_features, hidden_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_features)
    def forward(self, x):
        # x: (B, T, F)
        out, _ = self.lstm(x)
        return self.fc(out)

# training loop for teacher-forcing next-step prediction
def train_lstm_ar(model, data_loader, n_epochs=20, lr=1e-3, device='cpu'):
    model = model.to(device)
    opt = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    for epoch in range(n_epochs):
        tot_loss = 0.0
        model.train()
        for batch in data_loader:
            batch = batch.to(device)
            opt.zero_grad()
            out = model(batch)
            loss = loss_fn(out, batch)
            loss.backward()
            opt.step()
            tot_loss += loss.item() * batch.size(0)
        print(f'Epoch {epoch+1}/{n_epochs}  loss={tot_loss/len(data_loader.dataset):.6f}')
    return model

# sampling function: autoregressively generate a sequence from seed
def sample_lstm_ar(model, seed, length=128, device='cpu'):
    model = model.to(device).eval()
    seed = torch.tensor(seed, dtype=torch.float32).unsqueeze(0).to(device)  # (1,T0,F)
    generated = seed.clone()
    with torch.no_grad():
        for _ in range(length - seed.shape[1]):
            out = model(generated)  # (1,curT,F)
            next_step = out[:, -1:, :]
            generated = torch.cat([generated, next_step], dim=1)
    return generated.squeeze(0).cpu().numpy()

# create and (optionally) train LSTM AR
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device', device)
lstm_model = LSTMAutoRegressor(n_features=data.shape[2], hidden_size=128, n_layers=2)
# train quickly for demo (you can increase epochs)
lstm_model = train_lstm_ar(lstm_model, train_loader, n_epochs=8, lr=1e-3, device=device)

# save model
os.makedirs('../data', exist_ok=True)
torch.save({'model_state': lstm_model.state_dict(), 'mu': mu, 'std': std}, '../data/lstm_cme_gen.pt')
print('Saved LSTM model to ../data/lstm_cme_gen.pt')

# sample and inverse scale for quick preview
seed = data_scaled[0][:32]
gen = sample_lstm_ar(lstm_model, seed, length=128, device=device)
gen_inv = gen * std + mu
print('Generated sequence shape:', gen_inv.shape)
plt.plot(gen_inv[:,0])
plt.title('LSTM generated (inverse-scaled) - feature 0')
plt.show()


In [ ]:
# Cell 6: Simple LSTM-GAN (generator produces sequences, discriminator judges sequences)
class LSTMGenerator(nn.Module):
    def __init__(self, z_dim=32, seq_len=128, n_features=1, hidden=128, n_layers=1):
        super().__init__()
        self.seq_len = seq_len
        self.fc = nn.Linear(z_dim, seq_len * n_features)
        # optionally could use LSTM decoder, but for simplicity we map noise to sequence
        # a small LSTM for refinement
        self.lstm = nn.LSTM(n_features, hidden, n_layers, batch_first=True)
        self.head = nn.Linear(hidden, n_features)
    def forward(self, z):
        # z: (B, z_dim)
        B = z.size(0)
        x = self.fc(z).view(B, self.seq_len, -1)
        out, _ = self.lstm(x)
        return self.head(out)

class LSTMDiscriminator(nn.Module):
    def __init__(self, n_features=1, hidden=128, n_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(n_features, hidden, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        # use last hidden output
        last = out[:, -1, :]
        return self.fc(last)

# Loss and training loop
z_dim = 64
G = LSTMGenerator(z_dim=z_dim, seq_len=data.shape[1], n_features=data.shape[2], hidden=128).to(device)
D = LSTMDiscriminator(n_features=data.shape[2], hidden=128).to(device)
optG = optim.Adam(G.parameters(), lr=2e-4)
optD = optim.Adam(D.parameters(), lr=2e-4)
criterion = nn.BCEWithLogitsLoss()

real_label = 1.0
fake_label = 0.0

def train_gan(G, D, loader, n_epochs=30, z_dim=64, device='cpu'):
    for epoch in range(n_epochs):
        G.train(); D.train()
        tot_d_loss = 0.0; tot_g_loss = 0.0
        for real in loader:
            real = real.to(device)
            B = real.size(0)
            # train discriminator
            optD.zero_grad()
            z = torch.randn(B, z_dim, device=device)
            fake = G(z)
            # real
            out_real = D(real).view(-1)
            out_fake = D(fake.detach()).view(-1)
            lossD = criterion(out_real, torch.ones_like(out_real)) + criterion(out_fake, torch.zeros_like(out_fake))
            lossD.backward()
            optD.step()
            # train generator
            optG.zero_grad()
            z2 = torch.randn(B, z_dim, device=device)
            fake2 = G(z2)
            out_fake2 = D(fake2).view(-1)
            lossG = criterion(out_fake2, torch.ones_like(out_fake2))
            lossG.backward()
            optG.step()
            tot_d_loss += lossD.item()*B
            tot_g_loss += lossG.item()*B
        print(f'Epoch {epoch+1}/{n_epochs}  D_loss={tot_d_loss/len(loader.dataset):.6f}  G_loss={tot_g_loss/len(loader.dataset):.6f}')
    return G, D

# Train GAN (short run for demo)
G, D = train_gan(G, D, train_loader, n_epochs=25, z_dim=z_dim, device=device)
# Save GAN
torch.save({'G_state': G.state_dict(), 'D_state': D.state_dict(), 'mu': mu, 'std': std}, '../data/gan_cme_gen.pt')
print('Saved GAN models to ../data/gan_cme_gen.pt')

# Generate samples from GAN and plot
with torch.no_grad():
    z = torch.randn(4, z_dim, device=device)
    fake = G(z).cpu().numpy()
    fake_inv = fake * std + mu
    for i in range(fake_inv.shape[0]):
        plt.plot(fake_inv[i,:,0], alpha=0.9)
    plt.title('GAN-generated sequences (inverse-scaled)')
    plt.show()

# Save generated samples to CSV
gen_samples = fake_inv.reshape(fake_inv.shape[0], -1)
pd.DataFrame(gen_samples).to_csv('../data/gan_generated_cmes.csv', index=False)
print('Saved sample GAN-generated CSV to ../data/gan_generated_cmes.csv')


## Notes & Next steps

- The architectures and training loops are intentionally simple and meant as a starting point. For production-quality synthetic time-series generation consider:
  - TimeGAN or GANs with seq2seq LSTM decoders.
  - Conditional generation (condition on parameters or classes).
  - Adding gradient-penalty or WGAN-GP for stability.
  - More careful hyperparameter search, early stopping, and logging.

- To integrate with your project, replace the data loading cell with the exact reader for your CME files (the earlier notebooks reference `../data/cme*.txt`).
- Generated CSVs and model checkpoints are written to `../data/` so downstream notebooks can pick them up.

